In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp
import pythainlp.word_vector

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus, clean

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [3]:
file_name = 'แก้ปัญหาฝุ่น'

corpus = load_corpus('../data/' + file_name + '.txt')
print('Total documents', len(corpus))

Total documents 229


In [4]:
f = open('../data/tokenized/newmm/tokenized_' + file_name + '.txt')
# f = open('../data/tokenized/deepcut/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

### Preprocess Corpus

#### Remove Words

In [5]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.common.thai_stopwords()
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1858 words
filter frequent words: 797 words
filter letter words: 788 words
filter stop words: 543 words


In [6]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Doc2Vec

In [7]:
def token_to_string(tokenized_list):
    string = str(tokenized_list)
    string.replace(',', ' ')
    return clean(string)

doc2vec_corpus = []
for doc in tokenized_corpus:
    array = pythainlp.word_vector.sentence_vectorizer(token_to_string(doc))[0]
    if any(numpy.isnan(array)):
        print('Nan')
        array = numpy.zeros(300)
    doc2vec_corpus.append(array)
doc2vec_corpus = pandas.DataFrame(doc2vec_corpus, dtype=float)

/home/kanoonsantikul/Documents/senior-project/venv/lib/python3.6/site-packages/pythainlp/word_vector/__init__.py:98: RuntimeWarning: invalid value encountered in true_divide
  vec /= len(words)


Nan
Nan
Nan


#### Dimension Reduction

In [8]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [9]:
def get_document_vectors(corpus, value):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if value == 'bow':
        vectors = bow_corpus
    elif value == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        vectors = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(vectors):
        for id, score in doc:
            array[i, id] = score

        if value == 'bow' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [10]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [11]:
def eval_cluster(document_vectors, result):
    num_cluster = numpy.unique(result['predicted_label'])[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(document_vectors.iloc[i]))
        corpus_centroid.append(numpy.array(document_vectors.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [12]:
min_samples = 7
eps = 0.32
expand_rate = 0.05
epoch = 1

document_vectors = get_document_vectors(idx_corpus, 'bow')
# document_vectors = doc2vec_corpus

In [ ]:
eps = 0.75
model = NewSDC()
while eps <= 0.9:
    predicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(document_vectors, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    
    print(format(eps, ',.2f'), label_count)
    eps += 0.01
    
    if input('') != '':
        num_cluster = len(label_count)
        break

In [ ]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(document_vectors)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(document_vectors)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(document_vectors, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

#### Iterative New SDC

In [13]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(document_vectors, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(document_vectors, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

90.1933392375561
[85 22 22 41 15 11 15 18]
89.88273376704426
[91 20 22 41 13 11 14 17]
89.68308146523847
[91 20 22 41 15 11 13 16]


#### Grouping

In [14]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7]
7 4 0.3424632854694074
7 0 0.31329542399751625
[0, 1, 2, 3, 0, 5, 6, 0]


### Result

In [15]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [16]:
# result.to_csv('../data/results/iterative_new_sdc/' + file_name + '_2.csv', index=False)

# result = pandas.read_csv('../data/results/iterative_new_sdc/' + file_name + '.csv')

print(eval_cluster(document_vectors, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

89.68308146523847
[91 20 22 41 15 11 13 16]


In [17]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 5, 6), value=0)

2 เห็นด้วยที่สุดว่า ภาครัฐที่มีส่วนเกี่ยวข้องทำงานล่าช้า ไม่มีมาตรการทันที ไม่เห็นปลทั้งในระยะสั้นและระยะยาวค่ะ
4 หงุดหงิดมากอะ รร ลูกอยู่ตรงปทุมวัน เขตนี้ควันนรกมากเลยอะ ต้องพยายามให้ใส่หน้ากาก ไม่รู้จะช่วยได้แค่ไหน ให้ใส่แบบกันเยอะๆ นางก็บ่น แต่นางก็ใส่แหละ
5 ถูกต้องเลยครับรถเมล์ควรซื้อใหม่ยกทั้งระบบ อะไรที่ควรซื้อก็ควรทำนะครับ รถถัง เครื่องบิน เรือดำนำ ซื้อมาแสนแพง เอาออกมาใช้โชว์งานวันเด็ก...
8 >>ผลที่สะสมต่อไปอนาคตของชาติจะกลายเป็นผู้ป่วยที่ล้นระบบสาธารณสุขจนรับมือไม่ไหว >>การตื่นตัวจากภาครัฐน้อยมาก ทำให้คนไม่ตระหนักถึงอันตราย เด็กๆนั่งรถมอเตอร์ไซค์ไปโรงเรียนอย่างปกติ ผู้ปกครองคนที่ไม่ได้รับข่าวสารทางอินเตอร์เนตดูแต่ข่าวทางทีวีจะไม่รู้ว่าสถานการณ์ร้ายแรงและอันตรายต่อทุกคน >>>ความตื่นตัวมีน้อยมากๆ<<<
9 รร ประถมชายชื่อดังบาง รร ตอนพักกลางวันก็ยังคงไม่ให้เด็กขึ้นตึกเรียนเลยค่ะ วิ่งเล่นไปมากลางแจ้งนี่ล่ะ ฝ่ายกิจกรรม นร บอกว่าเป็นช่วงครูพัก ถ้าต้องการให้เด็กขึ้นเรียนให้ไปคุยตกลงกับครูประจำชั้นเองค่ะ งงใจผู้บริหาร รร สุดๆค่ะ
11 คิดถึง..วาทะ เด็ด ตอนน้ำท่วมกทม.อดีตผู้ว่า.ท่านบอก ให้ย้ายไ

In [18]:
w2 = Widget(result, 'comment')

ToggleButtons(index=3, options=(0, 1, 2, 3, 4, 5, 6, 7), value=3)

*0 รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
*1 หมอ ผมว่าสิ่งที่ควรทำแบบง่ายสุดในตอนนี้ คือ รณรงค์ให้ประชาชนใส่หน้ากากป้องกันตัวเองผมว่า ผู้ใหญ่ ผู้มีอำนาจในบ้านเรากลัวเสียหน้าที่จะยอมรับความจริงที่เกิดขึ้น มากกว่าห่วงสุขภาพประชาชนอธิบดีกรมควบคุมมลพิษที่มีหน้าที่โดยตรงออกมาหลายหนบอกว่าไม่วิกฤต ซึ่งมันสวนทางกับตาเราเห็นภาพวิว มองแทบไม่เห็นตึกสูง กลับบอกไม่อันตราย บอกตรงๆโกหกได้หน้าด้านมาก อาจเพราะในระยะสั้น มันมีผลกับคนที่ sensitive เช่น แสบตา คันคอ ไอ ซึ่งเลยทำให้เขาโกหกต่อได้ว่า เห็นไหม คนเป็นล้านมีคนเป็นไม่กี่คนเอง แต่ระยะยาวล่ะ โรคทางเดินหายใจ มะเร็งปอด มันพิสูจน์ตอนนี้ไม่ได้ เขาก็บอกไม่วิกฤตๆๆๆ ซ้ำร้าย คนเป็นหมอบางคนกลับบอกว่า ใส่ N95 ก็ไม่ช่วยอะไร มันทำให้คนสับสนไปหมด คือ คนที่ไม่รู้จริง ก็ควรนั่งนิ่งๆ ไม่ใช่ทำตัวเป็นประเด็นเรียกไลท์กัน มันยิ่งไปทำให้คนสับสนส่วนตัวเคยทำงานก่อสร้าง ก่อนที่จะรู้จักคำว่า pm2.5 ด้วยซ้ำ ก็เกือบ 20 ปีกว่าแล้ว เพราะเคยไปโคงานกับ บ ปูนแห่งนึง เลยเจอว่าพนักงานบัญชีในแพลนท์ปูนต้องตัดปอดท

In [19]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7), value=0)

2 ['เห็นด้วย', 'ภาครัฐ', 'ทำงาน', 'ไม่มี', 'มาตรการ', 'ไม่เห็น', 'ปล', 'ระยะสั้น', 'ระยะยาว']
5 ['รถเมล์', 'ซื้อ', 'ระบบ', 'ซื้อ', 'ทำ', 'ดำ', 'ซื้อ', 'แสน', 'แพง', 'ออกมา', 'งาน', 'เด็ก']
8 ['สะสม', 'ต่อไป', 'อนาคต', 'ชาติ', 'กลายเป็น', 'ผู้ป่วย', 'ระบบ', 'สาธารณสุข', 'รับมือ', 'ไม่ไหว', 'ตื่นตัว', 'ภาครัฐ', 'คน', 'อันตราย', 'เด็ก', 'นั่ง', 'โรงเรียน', 'ปกติ', 'ผู้ปกครอง', 'คน', 'ดู', 'ข่าว', 'ไม่รู้', 'สถานการณ์', 'อันตราย']
11 ['ตอน', 'กทม', 'ท่าน', 'ย้าย', 'ท่าน', 'ย้าย', 'ตำแหน่ง', '  ', 'คนเรา', 'ลูก', 'ทน', 'ดู', 'ลูก', 'หายใจ', 'ลำบาก', 'ไอ', 'อาศัย', 'ทน', 'ไม่ได้', 'หรอก', 'รัก', 'สภาพ', 'หายใจ', 'ฝุ่น', 'ท่าน', 'ทำ']
13 ['ประชาชน', 'ครึ่ง', 'ใส่หน้ากาก', 'แถม', 'คน', 'ใส่หน้ากาก', 'คน', 'ประหลาด', 'พูดว่า', 'ใส่หน้ากาก', 'เจ้า', 'ค่า', 'ธรรมชาติ', '  ', 'ล่ะ', 'เพลีย']
15 ['ยังมี', 'คน', 'จำนวน', 'พวกเรา', 'ตื่นตูม', 'อยู่เลย', 'ใส่หน้ากาก', 'โดน', 'เหมือน']
16 ['เห็นด้วย', 'แก้ปัญหา', 'ฝุ่น', 'ไม่ได้', 'ประชาชน', 'ดูแล', 'สุขภาพ', 'ออกมา', 'แจก', 'หน้ากาก', 'ฝุ่น', 'ดี', 'เ

In [20]:
seed = 0
compare = 0

a = numpy.array(document_vectors.iloc[seed]).reshape(1, -1)
b = numpy.array(document_vectors.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['รร', 'ประกาศ', 'หยุด', 'เรียน', 'ผู้บริหาร', 'โรงเรียน', 'ตัด', 'ประกาศ', '  ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
['รร', 'ประกาศ', 'หยุด', 'เรียน', 'ผู้บริหาร', 'โรงเรียน', 'ตัด', 'ประกาศ', '  ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
